In [ ]:
import warnings

warnings.filterwarnings("ignore")
import arviz as az
import numpy as np
import pandas as pd
from tb_incubator.input import load_param_info
from tb_incubator.constants import set_project_base_path
from tb_incubator.calibrate import calculate_xpert_scenario_outputs, calculate_detection_scenario_outputs, calculate_outputs_for_covid
import pickle

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")

In [ ]:
calib_out = project_paths["OUT_PATH"]
output_path = project_paths["OUTPUTS"]
file_suffix = "xpert_utilisation_improvement_p17_96"

In [ ]:
# extract data - only run for first time
idata = az.from_netcdf(calib_out / f'calib_full_out_{file_suffix}.nc')

In [ ]:
burnt_idata = idata.sel(draw=np.s_[2500:])
idata_extract = az.extract(burnt_idata, num_samples=1000)
inference_data = az.convert_to_inference_data(idata_extract.reset_index('sample'))
az.to_netcdf(inference_data, calib_out / f'inference_data_{file_suffix}.nc')

In [ ]:
# Load saved idata
idata = az.from_netcdf(calib_out / f'inference_data_{file_suffix}.nc')

In [ ]:
param_info = load_param_info()
fixed_params = param_info["value"]
covid_effects = {
    'detection_reduction':True
}

In [ ]:
basecase_xpert = calculate_xpert_scenario_outputs(fixed_params, idata, covid_effects=covid_effects)
with open(output_path / f'xpert_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_xpert, f)

In [ ]:
basecase_detection = calculate_detection_scenario_outputs(fixed_params, idata, covid_effects=covid_effects)
with open(output_path / f'detection_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_detection, f)